In [1]:
%load_ext autoreload
%autoreload 2

In [ ]:
from __future__ import annotations
import numpy as np

from mflex.simulate.simulate_boundary import dipole

from mhsflex.field2d import Field2dData
from mhsflex.field3d import Field3dData, calculate_magfield, lf3d, j3d
from mhsflex.vis import plot
from mhsflex.metrics import compare_field3d

from matplotlib import rc, colors
import matplotlib.pyplot as plt

from mhsflex.poloidal import (
    phi,
    phi_low,
    phi_hypgeo,
    dphidz_hypgeo,
    dphidz,
    dphidz_low,
)

from mhsflex.b3d import b3d

from mhsflex.fluxbalance import b3d_fb

rc("font", **{"family": "serif", "serif": ["Times"]})
rc("text", usetex=True)
c1 = (0.439, 0.788, 1.000)
c2 = (0.420, 0.502, 1.000)
c3 = "black"
c4 = (1.000, 0.224, 0.376)
c5 = (0.784, 0.231, 0.576)
c7 = (0.992, 0.251, 0.733)
c8 = (0.867, 0.871, 0.184)
c9 = (0.949, 0.922, 0.678)
c10 = (0.984, 0.455, 0.231)
c11 = (0.765, 0.835, 0.922)
c12 = (0.965, 0.694, 0.486)
c13 = (0.992, 0.584, 0.820)

In [3]:
nresol_x = 2000
nresol_y = nresol_x
nresol_z = nresol_x
xmin = 0.0
xmax = 20.0  # in units of 10^4 km, therefore corresponds to 20Mm
ymin = 0.0
ymax = 20.0
zmin = 0.0
zmax = 20.0
pixelsize_x = (xmax - xmin) / nresol_x
pixelsize_y = (ymax - ymin) / nresol_y
pixelsize_z = (zmax - zmin) / nresol_z

z0 = 2.0

nf_max = nresol_x

z_arr = np.arange(nresol_z) * (zmax - zmin) / (nresol_z - 1) + zmin
x_arr = np.arange(nresol_x) * (xmax - xmin) / (nresol_x - 1) + xmin
y_arr = np.arange(nresol_y) * (ymax - ymin) / (nresol_y - 1) + ymin
B_PHOTO = 500
data_bz = np.zeros((nresol_y, nresol_x))

for ix in range(0, nresol_x):
    for iy in range(0, nresol_y):
        x = x_arr[ix]
        y = y_arr[iy]
        data_bz[iy, ix] = dipole(x, y) * B_PHOTO

data2d = Field2dData(
    nresol_x,
    nresol_y,
    nresol_z,
    nresol_x,
    pixelsize_x,
    pixelsize_y,
    pixelsize_z,
    x_arr,
    y_arr,
    z_arr,
    data_bz,
)

In [4]:
a = 0.22
alpha = 0.05
b = 1.0
z0 = 2.0
deltaz = 0.2

In [5]:
length_scale = float(2.0)  # Normalising length scale for Seehafer

length_scale_x = 2.0 * nresol_x * data2d.px
length_scale_y = 2.0 * nresol_y * data2d.py

length_scale_x_norm = length_scale_x / length_scale
length_scale_y_norm = length_scale_y / length_scale

kx_arr = np.arange(data2d.nf) * np.pi / length_scale_x_norm  # [0:nf_max]
ky_arr = np.arange(data2d.nf) * np.pi / length_scale_y_norm  # [0:nf_max]
one_arr = 0.0 * np.arange(data2d.nf) + 1.0

ky_grid = np.outer(ky_arr, one_arr)  # [0:nf_max, 0:nf_max]
kx_grid = np.outer(one_arr, kx_arr)  # [0:nf_max, 0:nf_max]

# kx^2 + ky^2

k2_arr = np.outer(ky_arr**2, one_arr) + np.outer(one_arr, kx_arr**2)
k2_arr[0, 0] = (np.pi / length_scale_x_norm) ** 2 + (np.pi / length_scale_y_norm) ** 2
k2_arr[1, 0] = (np.pi / length_scale_x_norm) ** 2 + (np.pi / length_scale_y_norm) ** 2
k2_arr[0, 1] = (np.pi / length_scale_x_norm) ** 2 + (np.pi / length_scale_y_norm) ** 2

# Need k^2 > alpha^2/(1-2a) and k^2 > alpha^2

limit = (k2_arr - alpha**2) / (2 * k2_arr)

In [6]:
phi_asymp = np.zeros((nf_max, nf_max, nresol_z))
dphidz_asymp = np.zeros((nf_max, nf_max, nresol_z))
phi_hyper = np.zeros((nf_max, nf_max, nresol_z))
dphidz_hyper = np.zeros((nf_max, nf_max, nresol_z))

In [7]:
deltaz = 0.2

p_arr = (
    0.5 * deltaz * np.sqrt(k2_arr[0:nf_max, 0:nf_max] * (1.0 - a - a * b) - alpha**2)
)
q_arr = (
    0.5 * deltaz * np.sqrt(k2_arr[0:nf_max, 0:nf_max] * (1.0 - a + a * b) - alpha**2)
)

In [8]:
def check_runtime_pa():
    for iz, z in enumerate(z_arr):
        phi_asymp[:, :, iz] = phi(z, p_arr, q_arr, z0, deltaz)

In [ ]:
%timeit -r 1000 check_runtime_pa()

In [12]:
def check_runtime_ph():
    for iz, z in enumerate(z_arr):
        phi_hyper[:, :, iz] = phi_hypgeo(z, p_arr, q_arr, z0, deltaz)

In [ ]:
%timeit -r 1000 check_runtime_ph()

In [10]:
def check_runtime_dpa():
    for iz, z in enumerate(z_arr):
        dphidz_asymp[:, :, iz] = dphidz(z, p_arr, q_arr, z0, deltaz)

In [ ]:
%timeit -r 1000 check_runtime_dpa()

In [11]:
def check_runtime_dph():
    for iz, z in enumerate(z_arr):
        dphidz_hyper[:, :, iz] = dphidz_hypgeo(z, p_arr, q_arr, z0, deltaz)

In [ ]:
%timeit -r 1000 check_runtime_dph()

In [5]:
def check_runtime_b_asymptotic_seehafer():
    b3d(data2d, a, b, alpha, z0, deltaz, asymptotic=True, tanh=True)

In [6]:
def check_runtime_b_asymptotic_balanced():
    b3d_fb(data2d, a, b, alpha, z0, deltaz, asymptotic=True, tanh=True)

In [7]:
def check_runtime_b_nw_seehafer():
    b3d(data2d, a, b, alpha, z0, deltaz, asymptotic=False, tanh=True)

In [8]:
def check_runtime_b_nw_balanced():
    b3d_fb(data2d, a, b, alpha, z0, deltaz, asymptotic=False, tanh=True)

In [ ]:
%timeit -n 10 -r 10 check_runtime_b_asymptotic_seehafer()

In [ ]:
%timeit -n 10 -r 10 check_runtime_b_nw_seehafer()

In [ ]:
%timeit -n 10 -r 10 check_runtime_b_asymptotic_balanced()

In [ ]:
%timeit -n 10 -r 10 check_runtime_b_nw_balanced()